### RNN Example to Predict Stock Price
- Ref : https://towardsdatascience.com/a-comprehensive-guide-to-working-with-recurrent-neural-networks-in-keras-f3b2d5e2fa7f

### Note
- Since range of stock price can change in different time 
- ==> Fail to predict stock price
- ==> Better to process data to be difference / ratio

### Import

In [27]:
import datetime as dt
import numpy as np

from pandas_datareader import data
from sklearn.model_selection import train_test_split

In [31]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense

### Parameters

In [33]:
TRAIN_PERC = 0.9

In [34]:
WIN_SIZE = 50
EPOCH = 100

### Load Data

In [10]:
# Load stock price in the predetermined time range
df = data.DataReader('AMZN', 'yahoo', dt.datetime(2000, 1, 1), dt.datetime.now())
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
1999-12-31,79.3750,76.000000,79.3125,76.1250,7270700,76.1250
2000-01-03,89.5625,79.046875,81.5000,89.3750,16117600,89.3750
2000-01-04,91.5000,81.750000,85.3750,81.9375,17487400,81.9375
2000-01-05,75.1250,68.000000,70.5000,69.7500,38457400,69.7500
2000-01-06,72.6875,64.000000,71.3125,65.5625,18752000,65.5625


In [41]:
# Data Split
prices = df['Close']
train, test = train_test_split(list(prices), train_size=TRAIN_PERC, shuffle=False)

### Preprocessing

In [43]:
# Generate Training Sequences (Price_0 ~ Price_W-1 => Price_W)
X_train, y_train = [], []
for ind in range(len(train) - WIN_SIZE):
    X_train.append(train[ind:ind+WIN_SIZE])
    y_train.append(train[ind+WIN_SIZE])

In [44]:
# Reshape to 3D array (#Sequence, #Win, 1)
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = X_train[..., np.newaxis]

In [45]:
# Generate Testing Sequences (Price_0 ~ Price_W-1 => Price_W)
X_test, y_test = [], []
for ind in range(len(test) - WIN_SIZE):
    X_test.append(test[ind:ind+WIN_SIZE])
    y_test.append(test[ind+WIN_SIZE])

In [46]:
# Reshape to 3D array (#Sequence, #Win, 1)
X_test, y_test = np.array(X_test), np.array(y_test)
X_test = X_test[..., np.newaxis]

### Simple RNN

In [52]:
# Create Model
model_deep_simple_rnn = Sequential()
model_deep_simple_rnn.add(SimpleRNN(64, input_shape=(WIN_SIZE, 1), return_sequences=True))
model_deep_simple_rnn.add(SimpleRNN(32, return_sequences=True))
model_deep_simple_rnn.add(SimpleRNN(16))
model_deep_simple_rnn.add(Dense(8, activation='tanh'))
model_deep_simple_rnn.add(Dense(1, activation='linear'))

In [ ]:
# Train Model
model_deep_simple_rnn.compile(loss='mse', optimizer='adam', metrics=['mae'])
h_deep_simple_rnn = model_deep_simple_rnn.fit(X_train, y_train, epochs=EPOCH)

In [54]:
# Evaluate
model_deep_simple_rnn.evaluate(X_test, y_test)

15/15 [==============================] - 0s 5ms/step - loss: 3773446.2500 - mae: 1885.9846


[3773446.25, 1885.984619140625]

### LSTM

In [55]:
# Create Model
model_lstm = Sequential()
model_lstm.add(LSTM(64, input_shape=(WIN_SIZE, 1), return_sequences=True))
model_lstm.add(LSTM(32, return_sequences=True))
model_lstm.add(LSTM(16))
model_lstm.add(Dense(8, activation='tanh'))
model_lstm.add(Dense(1, activation='linear'))

In [ ]:
# Train Model
model_lstm.compile(loss='mse', optimizer='adam', metrics=['mae'])
h_lstm = model_lstm.fit(X_train, y_train, epochs=EPOCH)

In [ ]:
# Evaluate
model_lstm.evaluate(X_test, y_test)